In [17]:
import efel

from bluepy.v2 import Simulation

from bluepy.v2 import Circuit
from bluepy.v2 import Cell
from bluepy.v2.enums import Synapse

from os import path

import pandas as pd

import numpy as np

from matplotlib import pyplot as plt
import pickle


In [ ]:
##### INPUT FILES ##### 

# met_general

um_file_name = '/gpfs/bbp.cscs.ch/project/proj34/scratch/polina/um_withMet_2020111812.txt' # um with gen met: met_general : Calv no layers

feature_values_file_name = '/gpfs/bbp.cscs.ch/project/proj34/metabolismndam/out_data/efel_met_general_thal_f1.pickle'
feature_values_file_name2 = '/gpfs/bbp.cscs.ch/project/proj34/metabolismndam/out_data/efel_met_general_thal_f2.pickle'


BlueConfig_file_name = '/gpfs/bbp.cscs.ch/project/proj34/sh_ngvm/column/met_general/BlueConfig'

#########
circuit_config_file_name = '/gpfs/bbp.cscs.ch/project/proj64/circuits/O1.v6a/20181207/CircuitConfig'

In [ ]:
mean_freq = 'mean_frequency'

#features from Michiel

efel_feats1 = ['AP_amplitude','AP_height','AP_duration_half_width','mean_frequency',
 'AHP_depth','AHP_time_from_peak','voltage_base',
 'time_to_first_spike','time_to_last_spike','ISI_CV','ISI_log_slope',
 'inv_first_ISI',
 'inv_second_ISI',
 'inv_third_ISI', 
 'inv_fourth_ISI',
 'inv_fifth_ISI',
 'inv_last_ISI'
]

efel_feats2 = ['AHP_depth_abs', 'Spikecount','amp_drop_first_last','depolarized_base','irregularity_index', 'max_amp_difference','maximum_voltage','mean_AP_amplitude','peak_voltage','spike_half_width']

efel_feats = efel.getFeatureNames()

#np.setdiff1d(efel_feats,efel_feats1)

In [ ]:
current = ['current'] # for ATP prediction

In [16]:
np.setdiff1d(efel_feats,efel_feats1+efel_feats2)

array(['AHP1_depth_from_peak', 'AHP2_depth_from_peak',
       'AHP_depth_abs_slow', 'AHP_depth_diff', 'AHP_depth_from_peak',
       'AHP_slow_time', 'AP1_amp', 'AP1_begin_voltage', 'AP1_begin_width',
       'AP1_peak', 'AP1_width', 'AP2_AP1_begin_width_diff',
       'AP2_AP1_diff', 'AP2_AP1_peak_diff', 'AP2_amp',
       'AP2_begin_voltage', 'AP2_begin_width', 'AP2_peak', 'AP2_width',
       'AP_amplitude_change', 'AP_amplitude_diff',
       'AP_amplitude_from_voltagebase', 'AP_begin_indices',
       'AP_begin_time', 'AP_begin_voltage', 'AP_begin_width',
       'AP_duration', 'AP_duration_change',
       'AP_duration_half_width_change', 'AP_end_indices',
       'AP_fall_indices', 'AP_fall_rate', 'AP_fall_rate_change',
       'AP_fall_time', 'AP_phaseslope', 'AP_phaseslope_AIS',
       'AP_rise_indices', 'AP_rise_rate', 'AP_rise_rate_change',
       'AP_rise_time', 'AP_width', 'APlast_amp', 'APlast_width',
       'BAC_maximum_voltage', 'BAC_width', 'BPAPAmplitudeLoc1',
       'BPAPAmplit

In [5]:
um = pd.read_csv(um_file_name,header=None)

um_colnames = ["c_gid","mpi_rank","idxm","VNeu0", "m0", "h0", "n0", "Conc_Cl_out", "Conc_Cl_in", "Na0in", "K0out", "Glc_b", "Lac_b", "O2_b", "q", "Glc_ecs", "Lac_ecs", "O2_ecs", "O2_n", "O2_a", "Glc_n", "Glc_a", "Lac_n", "Lac_a", "Pyr_n", "Pyr_a", "PCr_n", "PCr_a", "Cr_n", "Cr_a", "ATP_n", "ATP_a", "ADP_n", "ADP_a", "NADH_n", "NADH_a", "NAD_n", "NAD_a", "ksi","ksi_ephys"]
print(len(um_colnames))

um.columns =  um_colnames

um = um.add_prefix('with_met_')


um = um.rename(columns={"with_met_c_gid":"c_gid",
                        "with_met_mpi_rank":"mpi_rank",
                        "with_met_idxm":"idxm"})

um.index = um['c_gid']

um_main = um[['c_gid','idxm','with_met_ATP_n']].copy()
print(len(um_main))

um_main = um_main.loc[um_main['with_met_ATP_n'] != " nan",:]

print(len(um_main))

um_main['with_met_ATP_n'] = um_main['with_met_ATP_n'].astype(float)

um_main = um_main.reset_index(drop=True)

um_main_g = um_main[['c_gid','with_met_ATP_n']].groupby('c_gid').mean()

um_main_g = um_main_g.reset_index()



40
114590
114531


In [99]:
np.max(um_main.idxm.unique())

4

In [50]:
um_idxm_stats = um_main[['c_gid','idxm']].groupby('c_gid').max()

In [57]:
um_idxm_stats[um_idxm_stats['idxm']!=np.max(um_main.idxm.unique())].index.to_list()

[32101.0,
 119772.0,
 124309.0,
 127270.0,
 141376.0,
 150010.0,
 150147.0,
 150855.0,
 154332.0,
 155039.0,
 155049.0,
 155412.0,
 157742.0,
 158280.0,
 158674.0,
 159320.0,
 160872.0]

In [58]:
um_main = um_main.loc[~um_main['c_gid'].isin(um_idxm_stats[um_idxm_stats['idxm']!=4].index.to_list())]

In [59]:
um_diff = {}

for idx,gid in enumerate(um_main['c_gid'].unique()):
    um_diff[gid] = ( (um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==0),'with_met_ATP_n'].values[0] - um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==1),'with_met_ATP_n'].values[0]) +
                    (um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==1),'with_met_ATP_n'].values[0] - um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==2),'with_met_ATP_n'].values[0]) +
                    (um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==2),'with_met_ATP_n'].values[0] - um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==3),'with_met_ATP_n'].values[0]) +
                    (um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==3),'with_met_ATP_n'].values[0] - um_main.loc[(um_main['c_gid']==gid)&(um_main['idxm']==4),'with_met_ATP_n'].values[0]) )/4
    
    if (idx%1000 ==0):
        print(idx)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


In [6]:
c = Circuit(circuit_config_file_name)
mc2_cells = c.cells.get({'$target': 'mc2_Column'}, properties=[Cell.X, Cell.Y, Cell.Z,Cell.SYNAPSE_CLASS,Cell.ETYPE,Cell.MTYPE,Cell.ME_COMBO,Cell.LAYER])

In [8]:
with open(feature_values_file_name, 'rb') as handle:
    feature_values = pickle.load(handle)

with open(feature_values_file_name2, 'rb') as handle:
    feature_values2 = pickle.load(handle)

In [61]:
fr1 = pd.DataFrame(feature_values)
fr2 = pd.DataFrame(feature_values2)

fr = pd.concat([fr1,fr2])

print((len(fr1)+len(fr2))/len(fr))

frt = fr.transpose()

frt = frt.reset_index()

frt = frt.rename(columns = {'index':'c_gid'})

frt['c_gid']=frt['c_gid'].astype(int)

frt.head()

1.0


,c_gid,AP_amplitude,AP_height,AP_duration_half_width,mean_frequency,AHP_depth,AHP_time_from_peak,voltage_base,time_to_first_spike,time_to_last_spike,...,inv_last_ISI,AHP_depth_abs,Spikecount,amp_drop_first_last,irregularity_index,max_amp_difference,maximum_voltage,mean_AP_amplitude,peak_voltage,spike_half_width
0,2,77.570482,15.966334,0.900000,1.003090,-17.377565,11.660000,-65.157120,14.8,4984.6,...,1.682086,-82.534685,5.0,-1.404843,107.233333,2.346976,17.293047,77.570482,15.966334,1.042538
1,3,75.449310,14.582276,0.925000,1.731190,-16.754121,12.137500,-65.189240,37.7,4621.1,...,1.401738,-81.943360,8.0,-0.171030,432.083333,3.365417,16.003441,75.449310,14.582276,1.063093
2,6,77.823770,15.600341,0.985714,1.406555,-19.657011,8.942857,-65.083435,45.7,4976.7,...,1.260239,-84.740446,7.0,-3.993122,796.820000,2.391552,17.374937,77.823770,15.600341,1.157945
3,11,49.694353,-11.811833,1.024176,18.422917,-13.315995,6.937363,-65.206215,3.3,4939.5,...,22.779043,-78.522210,91.0,5.784451,6.523596,1.191062,-7.720213,49.694353,-11.811833,1.206677
4,21,56.408144,-5.895187,1.025000,2.669336,-19.423728,7.816667,-65.168419,28.7,4495.5,...,2.549070,-84.592147,12.0,1.609798,185.580000,1.348218,-4.785717,56.408144,-5.895187,1.223249


In [68]:
um_diff1  = {int(k):v for k,v in um_diff.items()}

In [82]:
um_diff_df = pd.DataFrame.from_dict(um_diff1,orient='index',columns=['dATPfromum'])
um_diff_df = um_diff_df.reset_index()

um_diff_df = um_diff_df.rename(columns={'index':'c_gid'})

um_diff_df.head()

,c_gid,dATPfromum
0,5479,0.093661
1,109,0.218472
2,237,0.179878
3,2066,0.094640
4,531,0.093632


In [83]:

um_diff_df['c_gid']=um_diff_df['c_gid'].astype(int)


um_main_g['c_gid']=um_main_g['c_gid'].astype(int)




combo1 = pd.merge(um_main_g, frt, left_on='c_gid',right_on='c_gid',how="inner")
combo = pd.merge(um_diff_df, combo1, left_on='c_gid',right_on='c_gid',how="inner")



combo.index = combo['c_gid']
combo = combo.drop(columns='c_gid')

combo_feat = pd.merge(mc2_cells, combo, left_index=True,right_index=True, how='inner') # or onepercent instead of mc2_cells

combo_feat['with_met_ATP_n'] = combo_feat['with_met_ATP_n'].astype(float)

combo_feat.head()

,x,y,z,synapse_class,etype,mtype,me_combo,layer,dATPfromum,with_met_ATP_n,...,inv_last_ISI,AHP_depth_abs,Spikecount,amp_drop_first_last,irregularity_index,max_amp_difference,maximum_voltage,mean_AP_amplitude,peak_voltage,spike_half_width
2,484.474727,1933.685846,591.369003,INH,cNAC,L1_DAC,cNAC_L23NGC_L1_DAC_1_sm080529a1-5_idA_-_Scale_...,1,0.109598,1.213537,...,1.682086,-82.534685,5.0,-1.404843,107.233333,2.346976,17.293047,77.570482,15.966334,1.042538
3,426.301179,2001.898479,555.113190,INH,cNAC,L1_DAC,cNAC_L23NGC_L1_DAC_1_C270106A_-_Scale_x1.000_y...,1,0.214325,0.651347,...,1.401738,-81.943360,8.0,-0.171030,432.083333,3.365417,16.003441,75.449310,14.582276,1.063093
6,315.820475,2013.331452,540.811014,INH,cNAC,L1_DAC,cNAC_L4CHC_L1_DAC_1_sm080723a1-4_idC_-_Scale_x...,1,0.209361,0.669173,...,1.260239,-84.740446,7.0,-3.993122,796.820000,2.391552,17.374937,77.823770,15.600341,1.157945
11,445.350444,2032.703279,531.314828,INH,cNAC,L1_DAC,cNAC_L6NGC_L1_DAC_1_sm080723a1-4_idC,1,0.101183,0.367252,...,22.779043,-78.522210,91.0,5.784451,6.523596,1.191062,-7.720213,49.694353,-11.811833,1.206677
21,347.674335,1926.455702,440.353624,INH,bNAC,L1_DAC,bNAC_L23NGC_L1_DAC_1_sm100513a1-3_idC_-_Scale_...,1,0.213349,0.765225,...,2.549070,-84.592147,12.0,1.609798,185.580000,1.348218,-4.785717,56.408144,-5.895187,1.223249


In [84]:
# gen met thalam stim

sim = Simulation(BlueConfig_file_name)
report = sim.report('ATPConcAllCmps', source='h5')
atp = report.get()
atp = atp.reset_index()


atp_t = atp.transpose()

atp_t.columns = atp_t.loc['time',:]

atp_t.drop(atp_t.index[0], inplace=True)



#atp_t['mean'] = atp_t.mean(axis=1)

#atp_t['diff'] =  atp_t[0.0] - atp_t[4999.0]



atp_t['diff'] =  ((atp_t[0.0] - atp_t[999.0]) + (atp_t[1000.0] - atp_t[1999.0]) + (atp_t[2000.0] - atp_t[2999.0])+ (atp_t[3000.0] - atp_t[3999.0]) + (atp_t[4000.0] - atp_t[4999.0]))/5



#atp_t['diff'] = atp_t[0.0] - atp_t[999.0]

#atp_t['diff'] =  ((atp_t[0.0] - atp_t[999.0])/np.nanmean(atp_t[0:999]) +
#                  (atp_t[1000.0] - atp_t[1999.0])/np.nanmean(atp_t[1000:1999]) +
#                  (atp_t[2000.0] - atp_t[2999.0])/np.nanmean(atp_t[2000:2999]) +
#                  (atp_t[3000.0] - atp_t[3999.0])/np.nanmean(atp_t[3000:3999]) +
#                  (atp_t[4000.0] - atp_t[4999.0])/np.nanmean(atp_t[4000:4999]))/5

atp2 = pd.DataFrame(atp_t.loc[:,'diff'])
#atp2 = pd.DataFrame(atp_t.loc[:,['mean','diff']])

print(len(atp2))
atp2 = atp2.loc[~atp2['diff'].isna()]
print(len(atp2))

combo2 = pd.merge(atp2, combo_feat, left_index=True,right_index=True, how='inner')

# mean is mean ATP conc from ndam reports

#combo2 = combo2.rename(columns = {'mean':'meanATPconc'})
combo2 = combo2.rename(columns = {'diff':'deltaATPconc'})

combo2.head()


Have keys ({'FlushBufferScalar'}) that are not defined for section: Run
Ignoring unknown key: 'FlushBufferScalar'
`brain` module not found. Using experimental fallback implementation for H5 report access.


22918
22895


,deltaATPconc,x,y,z,synapse_class,etype,mtype,me_combo,layer,dATPfromum,...,inv_last_ISI,AHP_depth_abs,Spikecount,amp_drop_first_last,irregularity_index,max_amp_difference,maximum_voltage,mean_AP_amplitude,peak_voltage,spike_half_width
2,0.234067,484.474727,1933.685846,591.369003,INH,cNAC,L1_DAC,cNAC_L23NGC_L1_DAC_1_sm080529a1-5_idA_-_Scale_...,1,0.109598,...,1.682086,-82.534685,5.0,-1.404843,107.233333,2.346976,17.293047,77.570482,15.966334,1.042538
3,0.411729,426.301179,2001.898479,555.113190,INH,cNAC,L1_DAC,cNAC_L23NGC_L1_DAC_1_C270106A_-_Scale_x1.000_y...,1,0.214325,...,1.401738,-81.943360,8.0,-0.171030,432.083333,3.365417,16.003441,75.449310,14.582276,1.063093
6,0.429192,315.820475,2013.331452,540.811014,INH,cNAC,L1_DAC,cNAC_L4CHC_L1_DAC_1_sm080723a1-4_idC_-_Scale_x...,1,0.209361,...,1.260239,-84.740446,7.0,-3.993122,796.820000,2.391552,17.374937,77.823770,15.600341,1.157945
11,0.464305,445.350444,2032.703279,531.314828,INH,cNAC,L1_DAC,cNAC_L6NGC_L1_DAC_1_sm080723a1-4_idC,1,0.101183,...,22.779043,-78.522210,91.0,5.784451,6.523596,1.191062,-7.720213,49.694353,-11.811833,1.206677
21,0.396668,347.674335,1926.455702,440.353624,INH,bNAC,L1_DAC,bNAC_L23NGC_L1_DAC_1_sm100513a1-3_idC_-_Scale_...,1,0.213349,...,2.549070,-84.592147,12.0,1.609798,185.580000,1.348218,-4.785717,56.408144,-5.895187,1.223249


In [85]:
len(combo2)

22701

In [13]:
# script to calc efel features /gpfs/bbp.cscs.ch/project/proj34/metabolismndam/utils/getAllEfel.py

In [86]:
combo2.columns

Index(['deltaATPconc', 'x', 'y', 'z', 'synapse_class', 'etype', 'mtype',
       'me_combo', 'layer', 'dATPfromum', 'with_met_ATP_n', 'AP_amplitude',
       'AP_height', 'AP_duration_half_width', 'mean_frequency', 'AHP_depth',
       'AHP_time_from_peak', 'voltage_base', 'time_to_first_spike',
       'time_to_last_spike', 'ISI_CV', 'ISI_log_slope', 'inv_first_ISI',
       'inv_second_ISI', 'inv_third_ISI', 'inv_fourth_ISI', 'inv_fifth_ISI',
       'inv_last_ISI', 'AHP_depth_abs', 'Spikecount', 'amp_drop_first_last',
       'irregularity_index', 'max_amp_difference', 'maximum_voltage',
       'mean_AP_amplitude', 'peak_voltage', 'spike_half_width'],
      dtype='object')

#!pip install scikit-learn

In [88]:
from sklearn.linear_model import LinearRegression

In [89]:
print(len(combo2))
combo2 = combo2.dropna()
print(len(combo2))

22701
21554


In [90]:
#Multiple Linear Regression

model = LinearRegression().fit(combo2.loc[:,['layer','AP_amplitude', 'AP_height',
       'AP_duration_half_width', 'mean_frequency', 'AHP_depth',
       'AHP_time_from_peak', 'voltage_base', 'time_to_first_spike',
       'time_to_last_spike', 'ISI_CV', 'ISI_log_slope', 'inv_first_ISI',
       'inv_second_ISI', 'inv_third_ISI', 'inv_fourth_ISI', 'inv_fifth_ISI',
       'inv_last_ISI', 'AHP_depth_abs', 'Spikecount', 'amp_drop_first_last',
       'irregularity_index', 'max_amp_difference', 'maximum_voltage',
       'mean_AP_amplitude', 'peak_voltage', 'spike_half_width']], combo2.dATPfromum)

In [91]:
r_sq = model.score(combo2.loc[:,['layer','AP_amplitude', 'AP_height',
       'AP_duration_half_width', 'mean_frequency', 'AHP_depth',
       'AHP_time_from_peak', 'voltage_base', 'time_to_first_spike',
       'time_to_last_spike', 'ISI_CV', 'ISI_log_slope', 'inv_first_ISI',
       'inv_second_ISI', 'inv_third_ISI', 'inv_fourth_ISI', 'inv_fifth_ISI',
       'inv_last_ISI', 'AHP_depth_abs', 'Spikecount', 'amp_drop_first_last',
       'irregularity_index', 'max_amp_difference', 'maximum_voltage',
       'mean_AP_amplitude', 'peak_voltage', 'spike_half_width']], combo2.dATPfromum)

print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)

coefficient of determination: 0.4521876365373705
intercept: -1.9283922151210586
slope: [ 6.57766284e-03 -3.31146602e-04  2.04395064e-03  3.67003581e-05
  1.42631558e-03  1.00405391e-02  2.28332743e-03 -2.07498124e-02
 -1.94841691e-05  1.57923072e-06  6.48272337e-03 -9.89731423e-04
 -1.34528947e-04 -4.98236851e-05  3.70936912e-05 -2.21741784e-05
 -1.07891204e-06 -1.66808140e-04 -1.07092732e-02 -8.70713650e-05
 -1.20834691e-04  4.01564439e-07  8.20213784e-04 -3.39939992e-03
 -3.31146602e-04  2.04395064e-03  8.23013607e-05]


In [ ]:
#now can use this model to predict delta ATP in other sim runs by efel features

#y_pred = model.predict(x2)
#print('predicted response:', y_pred, sep='\n')